In [1]:
import spacy

In [2]:
spacy.__version__

'3.0.3'

In [3]:
nlp = spacy.load("en_core_web_trf")

In [15]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [13]:
nlp.meta['name']

'core_web_trf'

In [16]:
#nlp("hello there")._.trf_data

In [20]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))

In [22]:
import vectorian

from vectorian.importers import NovelImporter
from vectorian.session import LabSession
from vectorian.embeddings import SpacyTransformerEmbedding

# contextual embeddings
im = NovelImporter(nlp, embeddings=[SpacyTransformerEmbedding(nlp)])

#im = NovelImporter(nlp)
doc = im("/Users/arbeit/Wise Children Mini.txt")



Importing /Users/arbeit/Wise Children Mini.txt: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s]


In [24]:
doc._contextual_embeddings

{'core_web_trf/3.0.0': <vectorian.embeddings.InMemoryVectorsRef at 0x7ffc2afc3610>}

In [17]:
# note: you may also specify StackedEmbedding here.
session = LabSession(
    [doc],
    static_embeddings=[],
    token_mappings="default")

NameError: name 'LabSession' is not defined